In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz
from prefect import flow, task # Prefect flow and task decorators
import time

@task
def get_weather_data(district_context={'district':None, 'lat':None, 'lon':None}):
    # API endpoint and parameters
    WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"
    API_KEY = "acc0f6789b052a92716a21cbf9d2aa5a"  # Replace with your actual API key
    district = district_context['district']
    
    
    params = {
        "lat": district_context['lat'],
        "lon": district_context['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        time.sleep(2) 
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)


        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'district': district,
            'province':districts[district]['province'],
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp'],
            'main.temp_min': data['main']['temp_min'],
            'main.temp_max': data['main']['temp_max'],
            'main.feels_like': data['main']['feels_like'],
            'main.pressure': data['main']['pressure'],
            'main.humidity': data['main']['humidity'],
            'main.sea_level': data['main']['sea_level'],
            'main.grnd_level': data['main']['grnd_level'],
            'visibility': data['visibility'],
            'wind.speed': data['wind']['speed'],
            'wind.deg': data['wind']['deg']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict

    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

@task
def get_pollution_data(district_context={'district':None, 'lat':None, 'lon':None}):
    # API endpoint and parameters
    POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"
    API_KEY = "acc0f6789b052a92716a21cbf9d2aa5a"  # Replace with your actual API key
    district=district_context['district']

    params = {
        "lat": district_context['lat'],
        "lon": district_context['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(POLLUTION_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        time.sleep(2)
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)

        timestamp = datetime.now()
        
        # Create dictionary with required fields
        pollution_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'province':districts[district]['province'],
            'longtitude': data['coord']['lon'],
            'latitude': data['coord']['lat'],
            'components_co': data['list'][0]['components']['co'],
            'components_no': data['list'][0]['components']['no'],
            'components_no2': data['list'][0]['components']['no2'],
            'components_o3': data['list'][0]['components']['o3'],
            'components_so2': data['list'][0]['components']['so2'],
            'components_pm2_5': data['list'][0]['components']['pm2_5'],
            'components_pm10': data['list'][0]['components']['pm10'],
            'components_nh3': data['list'][0]['components']['nh3']
           
        }
        
        return pollution_dict
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None
@task
def get_combined_data(district='Mueang Chon Buri'):
    weather = get_weather_data(district)
    pollution = get_pollution_data(district)

    if weather is not None and pollution is not None:
        combined = {**weather, **pollution}
        return combined
    else:
        return None